In [1]:
import json, re, os
import pandas as pd
from langdetect import detect
# Set pandas to display 10 rows only
pd.options.display.max_rows = 10

## Data Import and Parsing
1. Read in the JSON files of each month of the year.
2. Combine all the files into a single dataframe for each year.

In [2]:
# Directory containing the JSON files
directory = '2014data'

# List to store DataFrames
dfs = []

# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json') and filename.startswith('RC_2014'):
        filepath = os.path.join(directory, filename)
        # Read JSON file into a DataFrame and append it to the list
        # data is written in lines separated by endlines like ‘\n‘, so we set lines=True
        df = pd.read_json(filepath, lines=True)
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
reddit_2014_df = pd.concat(dfs, ignore_index=True)

## Data selection

### Interested columns:
- **id** (String): The submission’s identifier, e.g., “5lcgjh”.
- **author** (String): The account name of the poster, e.g., “example username”.
- **subreddit** (String): Name of the subreddit that the submission is posted. Note that it excludes the prefix /r/. E.g., ’AskReddit’.
- **subreddit id** (String): The identifier of the subreddit, e.g., “t5 2qh1i”.
- **created_utc** (Integer): Timestamp indicating when the post was made.

Things to keep in mind:
**link_id** (String) -  The identifier of the ID of the parent submission of a comment.

## Data Cleaning
1. Select only the columns of interest.
2. Remove all rows with '[deleted]' body, body length of less than 50 words. Rename the body column into 'utterance'
4. Remove all the emojies and urls from the body
5. Convert the removal reason and distinguished columns into boolean values. If removal reason was not included in the dataset, make it a null value.
6. Parse through the body to identify the language and keep only English entries - This may not be a 100% accurate.
7. Create a csv file for the cleaned dataframe.
8. Repeat this process for each year.

In [10]:
df = reddit_2014_df[['parent_id','author','subreddit','created_utc','subreddit_id','body','id']]
# Create a dictionary mapping subreddit names to subreddit ids
subreddit_dict = dict(zip(df['subreddit_id'], df['subreddit']))
# drop all rows that contain '[deleted]'
df = df.drop(df[df['body'] == '[deleted]'].index)
# remove all body with length < 50 words
df = df[df['body'].str.findall(r'\w+').str.len() >= 50]

# Function to remove emojis using regex
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to remove URLs using regex
def remove_urls(text):
    text = re.sub(r'^ttps?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    url_pattern = re.compile(r'ttps?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Apply the functions to remove emojis and URLs to the 'text' column
df['body'] = df['body'].apply(remove_emojis)
df['body'] = df['body'].apply(remove_urls)

df.rename(columns = {'body':'utterance'}, inplace = True)
df.dtypes

# replace "null" values with False and all other values with True
df['distinguished'] = df['distinguished'].fillna(False).astype(bool)
# replace "null" values with False and all other values with True
df['removal_reason'] = df['removal_reason'].fillna(False).astype(bool)

In [5]:
# Use langdetect to retain only english entries
def detect_language(text):
    try:
        return detect(text)
    except:
        return None
df['language'] = df['utterance'].apply(detect_language)

df = df.drop(df[df['language'] != 'en'].index)

In [ ]:
# Remove language column
del df['language']

In [12]:
# Save dataframe to a csv
df.to_csv('RC_2014_counts.csv', index=False)